In [4]:
import numpy
import scipy
import os
import sys
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb

In [3]:
import math
import random

![title](img/htw_cr.png)

Aufgabe 1 anlegen eines Notebooks: 

- aus einen Gegebenen Polynom die Nullstellen und Polstellen berechnen 
- aus den Koeffizienten muss sich die Systemübertragungsfunktion darstellen lassen
- aus den Koeffizienten muss sich auch der Phasengang dargestellt werden.

Input: 
- Polynom

Output: 
- Nullstellen
- Polstellen
- Systemübertragungsfunktion
- Phasengang